# DAS Evolution Queries - Sentences Dataset

This notebook demonstrates how to use Evolution-based queries with Hyperon MeTTa and DAS. Evolution queries use correlation analysis to find patterns across multiple generations of results.


## Load Sentences Dataset (Optional)

If not already loaded, use das-cli to load a sentences dataset containing 100K sentences with 10 words each (words starting with letters a-e).


In [ ]:
# !das-cli metta load /tmp/100K_sentences_10_words_a-e.metta

## Setup

Initialize `hyperon` MeTTa environment and create a helper function to run MeTTa programs.


In [ ]:
import hyperon

metta = hyperon.MeTTa()
def run(program='!(+ 1 2)'):
    for result in metta.run(program):
        for child in result:
            print(child)

## Import DAS Module

Import the DAS module into the MeTTa environment.


In [ ]:
run('!(import! &self das)')

## Connect to DAS

Bind a DAS connection to `&das` space. The first parameter specifies a client's host and port range (47000-47999) and the second must be a known peer address (eg. Query Agent at localhost:40002).


In [ ]:
run('!(bind! &das (new-das! (localhost:47000-47999) (localhost:40002)))')

## Simple Query: Find Words in Sentence

Find all words contained in a specific sentence. This is a basic pattern match to verify the dataset is loaded.


In [ ]:
run('!(match &das (Contains (Sentence "bce ecc dcb ced bbb bca bce cad eba ede") $W) $W)')

## Define Helper Functions and Evolution Parameters

Define utility functions for text analysis:
- **String operations**: length calculation, character counting, space removal
- **Fitness function (ff)**: Calculates the frequency of a letter in a sentence
- **Query definition**: Pattern to search for sentences containing the word "bbb"
- **Correlation parameters**: Define how evolution should correlate results across generations


In [ ]:
run('''
(= (str-length $s) (* ((py-dot "" len) $s) 1.0))
(= (count-letters $s $c) (* ((py-dot $s count) $c) 1.0))
(= (remove-spaces $s) ((py-dot $s replace) " " ""))
(= (prep-sentence $s) (remove-spaces (index-atom $s 1)))
(= 
  (ff $s $c) 
  (/ 
    (count-letters (prep-sentence $s) $c) 
    (str-length (prep-sentence $s))
  )
)

(= 
  (query) 
  (or 
    (Contains $sentence1 (Word "bbb")) 
    (Contains $sentence1 (Word "bbb"))
  )
)

(=
  (correlation-queries)
  (
    (Contains $sentence1 $word1)
  )
)

(=
  (correlation-replacements)
  (
    (sentence1 sentence1)
  )
)

(=
  (correlation-mappings)
  (
    (sentence1 word1)
  )
)
''')

## Check Current Evolution Parameters

Display the current DAS evolution parameters to see default settings.


In [ ]:
run('!(das-get-params!)')

## Set Maximum Generations

Configure the evolution to run for 5 generations only. Each generation refines the search based on correlation analysis.


In [ ]:
run('!(das-set-param! (max_generations 5))')

## Run Evolution Query

Execute an evolution-based query that:
1. Searches for sentences containing "bbb"
2. Analyzes the frequency of letter "c" in matching sentences
3. Evolves over 5 generations to find sentences with optimal "c" frequency
4. Uses correlation mappings to refine results across generations


In [ ]:
run('''
!(match 
  &das 
  (
    EVOLUTION 
    (!(query) !(correlation-queries) !(correlation-replacements) !(correlation-mappings))
    ((ff $sentence1 "c")) 
  ) 
  $sentence1
)
''')

## Test Fitness Function - Example 1

Calculate the frequency of letter "c" in a sample sentence to verify the fitness function works correctly.


In [ ]:
run('!(ff (Sentence "abc ada ccc eea bbb acc bcc cde cee cbd") "c")')

## Test Fitness Function - Example 2

Calculate the frequency of letter "c" in another sample sentence for comparison.


In [ ]:
run('!(ff (Sentence "cbd edd bde dda cde eaa cac eec cad bbb") "c")')